In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import requests
import re
from bs4 import BeautifulSoup
import json

In [2]:
def findQuantity(text):
    unit="(g|kg|l|ml| litro)"
    x = re.search("[0-9,]+"+unit, text, re.IGNORECASE)

    return x.group()

In [3]:
def findRelPrice(price, quantity):
    qt = int(''.join(filter(str.isdigit, quantity)))
    unit = ''.join(filter(str.isalpha, quantity))

    relPrice = float(price)/qt

    if unit == 'g' or unit == 'ml':
        relPrice*=1000
    
    return str(round(relPrice, 2)) +  ' R$/' + ("kg" if unit == 'g' else 'L')

In [4]:
baseURL = "https://paodeacucar.resultspage.com/search?p=Q&ts=json-full&lot=json&w={}&cnt=12&ref=www.paodeacucar.com&ua=Mozilla%2F5.0%20(Windows%20NT%2010.0%3B%20Win64%3B%20x64)%20AppleWebKit%2F537.36%20(KHTML%2C%20like%20Gecko)%20Chrome%2F81.0.4044.138%20Safari%2F537.36&ep.selected_store=501"

In [5]:
def fetchItems(products):
    results = pd.DataFrame(columns=['searchKey', 'productName', 'price', 'quantity', 'relPrice'])

    for key in products:
        url = baseURL.format(key.lower().replace(' ', '+'))
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        items = json.loads(str(soup))['results']
        for i, item in enumerate(items):
            qt = findQuantity(item['title'])
            price = float(item['price'].replace(',','.'))
            if int(item['instock']) > 0:
                results.loc[key.lower().replace(' ', '_') + '_' + str(i)] = [key, item['title'], price, qt, findRelPrice(price, qt)]

    return results

In [6]:
def getSuggested(results):
    suggested = results.drop_duplicates(subset=['searchKey'], keep='first')
    return suggested

In [7]:
def getMinPrice(results):
    min_price = results.groupby('searchKey').min()
    return min_price

In [8]:
def calculatePrices(chosen):
    chosen['amount'] = 0
    print("Quantos de cada um dos itens abaixo você deseja comprar?")
    for i,item in chosen.iterrows(): 
        print(item['productName'] + " (R$ " + str(item['price']) + ")")
        chosen.loc[i,'amount'] = int(input())

    chosen['totalPrice'] = chosen.amount*chosen.price
    return chosen